In [31]:
import pandas as pd
import requests
import json
import time
import openai

In [32]:
only_prompts = pd.read_json("../dataset/only_prompts.json")[0]
N_PROMPTS_TO_PROCESS = 5

In [33]:
BASE_URL = "http://213.173.102.136:10400"

VERIFY_TOKEN = "SjhSXuEmZoW#%SD@#nAsd123bash#$%&@n"
def get_evaluation(json_payload):
    response = requests.post(BASE_URL, json=json_payload)
    return response.json()

In [61]:
def get_model_response(prompt, model_name=None):
    start_time = time.time()
    time.sleep(0.1)
    response = "error"
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time, response
def generate_evaluation_json_list(prompts_df):
    evaluations_json_list = []
    for prompt in prompts_df[:N_PROMPTS_TO_PROCESS]:
        elapsed_time, text_response = get_model_response(prompt)
        json_payload = {
            "verify_token": VERIFY_TOKEN,
            "prompt": prompt,
            "responses": [text_response,]
        }
        evaluation_json = get_evaluation(json_payload)
        evaluation_json["time_elapsed_in_seconds"] = elapsed_time
        evaluations_json_list.append(evaluation_json)
    return evaluations_json_list
def generate_evaluation_df_from_json_list(evaluations_json_list):
    evaluation_data_list = []
    for evaluation_json in evaluations_json_list:
        rewards_details = evaluation_json["reward_details"]
        reciprocate_reward_model=rewards_details["reciprocate_reward_model"][0]
        relevance_filter=rewards_details["relevance_filter"][0]
        rlhf_reward_model=rewards_details["rlhf_reward_model"][0]
        evaluation_data_list.append(
            [
                reciprocate_reward_model,
                relevance_filter,
                rlhf_reward_model,
                evaluation_json["rewards"][0],
                evaluation_json["time_elapsed_in_seconds"]
            ]
        )
    evaluation_df= pd.DataFrame(evaluation_data_list, columns=["reciprocate_reward_model", "relevance_filter", "rlhf_reward_model", "rewards", "time_elapsed_in_seconds"])
    return evaluation_df